# Getting data for southern spain

In this notebook we will grab some data from southern Spain using a python package called `lsdviztools`. 
We also will do a little bit of topographic analysis using **lsdtopotools**.

We are assuming you are on a Notable notebook via the University of Edinburgh's Learn pages. This already has **lsdtopotools** installed. If this is not the case, you will need to use conda to install that yourself. 

## Get the right python packages

In [ ]:
!pip install lsdviztools

In [ ]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio
import lsdviztools.lsdmapwrappers as lsdmw
import pandas as pd
import geopandas as gpd
import cartopy as cp
import cartopy.crs as ccrs
import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np

## Now grab some data

First, we need to grab some data. We use a tool in `lsdviztools.lsdbasemaptools` called the `ot_scraper` (`ot` is for opentopography.org).

You can tell this what sort of data you want (most people will use the SRTM 30 metre data) and you also tell it the lower left and the upper right corners, using latitude and longitude. You can get these from goole earth by right clicking on the map and selecting "what's here". In this example below, I will just get a small area near Sorbas. But you might expand your search area. 

In [ ]:
Sorbas_DEM = bmt.ot_scraper(source = "SRTM30",longitude_W = -2.2227035399246113, longitude_E = -1.961614783210036, 
                            latitude_S = 37.026746017152774, latitude_N = 37.17085907663139,prefix = "Sorbas")
Sorbas_DEM.print_parameters()
Sorbas_DEM.download_pythonic()

That just downloaded a .tif file, which you could look at in a GIS. 
You can also look at the raw data here in python using something called `rasterio`. 

But we will use some tools developed at the University of Edinburgh, called **lsdtopotools**, to look at the data. 

To do that, we need to convert the data into a format **lsdtopotools** can understand. 

In [ ]:
DataDirectory = "./"
RasterFile = "Sorbas_SRTM30.tif"
gio.convert4lsdtt(DataDirectory, RasterFile,minimum_elevation=0.01,resolution=30)

## Now we do some topographic analysis and look at the data

We will extract some topogaphic metrics using `lsdtopotools`. 
This is already installed on the Noteable GeoScience Notebooks.

The `lsdtt_parameters` are the various parametes that you can use to run an analysis. We will discuss these later. For now, we will just follow this recipie. 

In [ ]:
lsdtt_parameters = {"write_hillshade" : "true",  
                    "surface_fitting_radius" : "60",
                    "print_slope" : "true"}
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = "Sorbas_SRTM30_UTM",
                                 write_prefix= "Sorbas_SRTM30_UTM",
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()


In [ ]:
lsdtt_drive.run_lsdtt_command_line_tool()

## Plot some data

We are now going to do some simple plots using a mapping package that we put together. There are more general ways to visualise data, but this makes pretty pictures quickly.  

In [ ]:
%matplotlib inline
Base_file = "Sorbas_SRTM30_UTM"
DataDirectory = "./"
this_img = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth", 
                                 save_fig=False, size_format="geomorphology")

We can also plot the slope map

In [ ]:
Base_file = "Sorbas_SRTM30_UTM"
Drape_prefix = "Sorbas_SRTM30_UTM_SLOPE"
DataDirectory = "./"
img_name2 = lsdmw.SimpleDrape(DataDirectory,Base_file, Drape_prefix, 
                              cmap = "bwr", cbar_loc = "right", 
                              cbar_label = "Gradient (m/m)",
                              save_fig=False, size_format="geomorphology",
                              colour_min_max = [0,1.25])

## Get some channel profiles

Okay, we will now run a different analysis. We will get some channel profiles. 

In [ ]:
lsdtt_parameters = {"print_basin_raster" : "true",
                    "print_chi_data_maps" : "true"}
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = "Sorbas_SRTM30_UTM",
                                 write_prefix= "Sorbas_SRTM30_UTM",
                                 read_path = "./",
                                 write_path = "./",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()

In [ ]:
lsdtt_drive.run_lsdtt_command_line_tool()

Now we are going to read in the channel data using geopandas

In [ ]:
df = pd.read_csv("Sorbas_SRTM30_UTM_chi_data_map.csv")